# 资讯分析师

## 准备工作

pip安装 `akshare` 和 `langchain` 等必需的库：

In [18]:
%pip install --upgrade --quiet pandas akshare lark langchain-core langchain-openai langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


获取股票主营介绍数据：

In [ ]:
import pandas as pd
import akshare as ak

# 沪深京 A 股实时行情数据
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
# 股票对应的主营介绍数据
stock_zyjs_ths_dfs = []
for stock_code, stock_name in zip(stock_zh_a_spot_em_df['代码'], stock_zh_a_spot_em_df['名称']):
    try:
        stock_zyjs_ths_df = ak.stock_zyjs_ths(stock_code)
        stock_zyjs_ths_df['股票名称'] = stock_name
        stock_zyjs_ths_dfs.append(stock_zyjs_ths_df)
    except Exception as e:
        print(f"Error processing stock {stock_code}: {e}")
        continue
stock_zyjs_ths_df = pd.concat(stock_zyjs_ths_dfs)
stock_zyjs_ths_df.to_csv('data/stock_zyjs_ths.csv', index=False, quoting=1)

加载股票主营介绍数据：

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path='data/stock_zyjs_ths.csv',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    },
    metadata_columns=['股票代码', '股票名称']
)

documents = loader.load()
print(f"Loaded {len(documents)} documents")

Loaded 5662 documents


embedding股票主营介绍数据并存储到向量数据库：

In [ ]:
import os
import pandas as pd

from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
)

vector_store = Chroma(
    collection_name="stock_zyjs_ths",
    embedding_function=embeddings,
    persist_directory="data/verctor_store.db",
)

ids = pd.read_csv('data/stock_zyjs_ths.csv')['股票代码'].astype(str).tolist()

for i in range(0, len(ids), 100):
    print(f"Adding documents {i} to {i+100}")
    partial_ids = ids[i:i+100]
    partial_documents = documents[i:i+100]
    # vector_store.add_documents(documents=partial_documents, ids=partial_ids)
    vector_store.update_documents(documents=partial_documents, ids=partial_ids)

## 资讯分析

连接向量数据库：

In [1]:
import os

from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
)

vector_store = Chroma(
    collection_name="stock_zyjs_ths",
    embedding_function=embeddings,
    persist_directory="data/verctor_store.db",
)

定义检索器：

In [9]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_CHAT_API_VERSION"],
)

metadata_field_info = [
    AttributeInfo(
        name="股票代码",
        description='股票代码',
        type="string",
    ),
    AttributeInfo(
        name="股票名称",
        description="股票名称",
        type="integer",
    ),
]

document_content_description = "股票主营介绍"

retriever = SelfQueryRetriever.from_llm(
    llm,
    vector_store,
    document_content_description,
    metadata_field_info,
)


In [14]:
retriever.batch(
    [
        "股票上海机场的主营业务是什么？",
        "股票600009的主营业务是什么？",
    ],
)

[[Document(metadata={'row': 1134, 'source': 'data/stock_zyjs_ths.csv', '股票代码': '600009', '股票名称': '上海机场'}, page_content='主营业务: 从事航空运输地面服务及其他相关业务。\n产品类型: 航空及相关服务\n产品名称: 航空及相关服务\n经营范围: 国内外航空运输企业及旅客提供地面保障服务，经营出租机场内航空营业场所、商业场所和办公场所，国内贸易（除专项规定）；广告经营，经营其它与航空运输有关的业务；货运代理；代理报验；代理报关；长途客运站（限分支机构经营）；综合开发，经营国家政策许可的其它投资项目，停车场管理及停车延伸服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）。')],
 [Document(metadata={'row': 1134, 'source': 'data/stock_zyjs_ths.csv', '股票代码': '600009', '股票名称': '上海机场'}, page_content='主营业务: 从事航空运输地面服务及其他相关业务。\n产品类型: 航空及相关服务\n产品名称: 航空及相关服务\n经营范围: 国内外航空运输企业及旅客提供地面保障服务，经营出租机场内航空营业场所、商业场所和办公场所，国内贸易（除专项规定）；广告经营，经营其它与航空运输有关的业务；货运代理；代理报验；代理报关；长途客运站（限分支机构经营）；综合开发，经营国家政策许可的其它投资项目，停车场管理及停车延伸服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）。')]]

In [5]:
import akshare as ak

from IPython.display import display, HTML

df = ak.stock_info_global_sina()
display(HTML(df.head(3).to_html()))

,时间,内容
0,2025-01-07 22:47:38,市场消息：特朗普请求纽约州上诉法院在宣判前驳回封口费案。
1,2025-01-07 22:46:44,美股医药股上涨，modena涨7.5%，NOVAVAX涨12.4%，辉瑞涨1.6%，美国上市的CUREVAC股价涨10.6%。
2,2025-01-07 22:45:35,【顺威股份：没有生产储能电站、数据中心领域等相关产品】 顺威股份(002676)1月7日在互动平台表示，公司目前没有生产关于储能电站、数据中心领域等相关产品。
